In [7]:
# prompt: pip install wandb

!pip install wandb


In [2]:
# prompt: Connect to wandb using the key stored in the variable wandb_key and return list of runs

import wandb
wandb.login()



wandb: Currently logged in as: shahar-r-oriel (common-sense-makers). Use `wandb login --relogin` to force relogin


True

In [9]:
#list names of projects by common-sense-makers
entity="common-sense-makers"
projects = wandb.api.list_projects(entity=entity)
for project in projects:
    print(project["name"])

desci-sense-notebooks_notebooks
uncategorized
pytorch-intro
basic-intro
st-app-multi-v1
st-demo-sandbox
st_demo-multi_v1_testing
st_demo-multi_v1
data_aggregation
st_demo-v0.2


In [10]:
project = "st-app-multi-v1"




In [ ]:
#log data as json
import wandb
import pandas as pd

# Assuming you have a DataFrame named df with your data
df = pd.read_csv("/content/st_data.csv")

wandb.init(project="testing")
artifact = wandb.Artifact("labeled_data_v0", type="dataset")

# Save the DataFrame to a JSON file
json_path = "/content/st_data.json"
df.to_json(json_path, orient="records")

# Add the JSON file to the artifact
artifact.add_file(json_path, name="st_data.json")

# Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()


In [13]:
#log dataset csv as wandb.Table object
#which is also JSON but has nice display
import wandb
import pandas as pd
import sys
from pathlib import Path
import os

# Assuming you have a DataFrame named df with your data
#notebook_path = Path(os.getcwd())
#sys.path.append(str(notebook_path.parents[1]))

df = pd.read_csv("/Users/shaharorielkagan/Documents/Python/desci-sense/data/output_data.csv")

wandb.init(project="testing")
artifact = wandb.Artifact("labeled_data_v0", type="dataset")

# Create a wandb.Table from the Pandas DataFrame
table = wandb.Table(dataframe=df)

# Add the wandb.Table to the artifact
artifact.add(table, "labeled_data_table")

# Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


This block imports a table from an artifact, whose path we must specify, and saves only the 'Text' and 'True Label' filed in a pd data frame. So later, we can evaluate predictions of models over the text entries. Remark, no reason to not get the full table as being done below.

In [29]:
#download a table from datasetartifact
#save it in a dataframe
path = "common-sense-makers/testing/labeled_data_v0:v5"

import wandb
from pathlib import Path
import json
import pandas as pd


api = wandb.Api()
artifact = api.artifact(path)

#download path to table
a_path = artifact.download()
table_path = Path(f"{a_path}/labeled_data_table.table.json")
raw_data = json.load(table_path.open())

#put it in a dataframe
x = raw_data["columns"].index('Text')
y = raw_data["columns"].index('True Label')

#Add id

try:
  rows = [{'Text': raw_data["data"][i][x], 'True Label': raw_data["data"][i][y]} for i in range(len(raw_data["data"]))]
except Exception as e:
  print(f"Exception occurred: {e}")

df = pd.DataFrame(rows)

print(df)


wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


                                                 Text  \
0   Huge congrats to the team on this release. Hop...   
1   Huge congrats to the team on this release. Hop...   
2   "Building a Data Pipeworks for Democratic AI: ...   
3   I'll endorse this and double down: there's no ...   
4   "Ultimately, privacy law’s primary goal should...   
5   The “singularity” to worry about, say ⁦@glenwe...   
6   Join my lab as a postdoc in the Cognitive Scie...   
7   Since writing this, I've found others studying...   
8   Nice perspective paper on fixing activities be...   
9   Some of my dissertation work was recently publ...   
10  A new study published by PNAS adds more detail...   
11  An explanation for why GPT-4 is degrading:\n\n...   
12  This looks very interesting:\n"Secularism as M...   
13  Test sets have 100s or even 1000s of examples\...   
14  From the abstract I can tell this is going to ...   
15  Man, this paper is good: https://philpapers.or...   
16  Good and balanced article: 

Get the entire table

In [72]:
#download a table from datasetartifact
#save it in a dataframe
path = "common-sense-makers/testing/labeled_data_v0:v5"

import wandb
from pathlib import Path
import json
import pandas as pd


api = wandb.Api()
artifact = api.artifact(path)

#download path to table
a_path = artifact.download()
table_path = Path(f"{a_path}/labeled_data_table.table.json")
raw_data = json.load(table_path.open())

#Add id

try:
  rows = [dict(zip(raw_data['columns'],raw_data['data'][i])) for i in range(len(raw_data["data"]))]
except Exception as e:
  print(f"Exception occurred: {e}")

df = pd.DataFrame(rows)

print(df)

wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


                                                 User  \
0                                                 rtk   
1                                        Nick Vincent   
2                                        Nick Vincent   
3                    p(Alex Kiefer | everything else)   
4                                       Abeba Birhane   
5                                         Nils Gilman   
6                                      Tania Lombrozo   
7                                  Elizabeth McDaniel   
8                                   Kording —-& Lab 🦖   
9                                    Zr. Nabu Kudurru   
10                                 J. Nathan Matias 🦣   
11  rtk - https://twitter.com/ChombaBupe/status/17...   
12                                         Tim Howles   
13                               ♻️ Leshem Choshen ♻️   
14  rtk - https://mastodon.social/@jonny@neuromatc...   
15                                   Mx. Aria Stewart   
16   Filip Piekniewski🌻 🐘:@fili

## Binarize and evaluate
Here we will get a table with predictions and calculate scores. No need to do this if we get the entire table 


In [51]:
#same cell as above only with "Predicted_Labels"
#download a table from datasetartifact
#save it in a dataframe
path = "common-sense-makers/testing/labeled_data_v0:latest"

import wandb
from pathlib import Path
import json
import pandas as pd


api = wandb.Api()
artifact = api.artifact(path)

#download path to table
a_path = artifact.download()
table_path = Path(f"{a_path}/labeled_data_table.table.json")
raw_data = json.load(table_path.open())

#put it in a dataframe
x = raw_data["columns"].index('Text')
y = raw_data["columns"].index('True Label')
z = raw_data["columns"].index('Predicted Label')

try:
  rows = [{'Text': raw_data["data"][i][x], 'True Label': raw_data["data"][i][y], 'Predicted Label': raw_data['data'][i][z]} for i in range(len(raw_data["data"]))]
except Exception as e:
  print(f"Exception occurred: {e}")

df = pd.DataFrame(rows)

print(df)

wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


                                                 Text  \
0   Huge congrats to the team on this release. Hop...   
1   Huge congrats to the team on this release. Hop...   
2   "Building a Data Pipeworks for Democratic AI: ...   
3   I'll endorse this and double down: there's no ...   
4   "Ultimately, privacy law’s primary goal should...   
5   The “singularity” to worry about, say ⁦@glenwe...   
6   Join my lab as a postdoc in the Cognitive Scie...   
7   Since writing this, I've found others studying...   
8   Nice perspective paper on fixing activities be...   
9   Some of my dissertation work was recently publ...   
10  A new study published by PNAS adds more detail...   
11  An explanation for why GPT-4 is degrading:\n\n...   
12  This looks very interesting:\n"Secularism as M...   
13  Test sets have 100s or even 1000s of examples\...   
14  From the abstract I can tell this is going to ...   
15  Man, this paper is good: https://philpapers.or...   
16  Good and balanced article: 

In [73]:
import re
# Assuming each label is a single word and there are no spaces in labels
# This will remove all non-word characters and split the string into words
df['True Label'] = df['True Label'].apply(lambda x: re.sub(r'\W+', ' ', x).split())

# Replace None with an empty list in 'Predicted label'
df['Predicted Label'] = df['Predicted Label'].apply(lambda x: '' if pd.isnull(x) else x)    
df['Predicted Label'] = df['Predicted Label'].apply(lambda x: re.sub(r'\W+', ' ', x).split())



In [82]:
print(df['True Label'])

0                                  [announce]
1                                  [announce]
2                                  [announce]
3                [recommendation, discussion]
4                                     [quote]
5                           [quote, announce]
6                                       [job]
7                      [question, discussion]
8                [recommendation, discussion]
9                                  [announce]
10                                 [announce]
11                        [discussion, quote]
12                  [recommendation, reading]
13                      [reading, discussion]
14                                  [reading]
15                   [review, recommendation]
16          [review, recommendation, reading]
17                  [reading, recommendation]
18                                 [announce]
19                                 [announce]
20                     [announce, discussion]
21                [recommendation,

In [75]:
from sklearn.preprocessing import MultiLabelBinarizer

# Assume df['True label'] and df['Predicted label'] are your true and predicted labels
mlb = MultiLabelBinarizer()

# Binarize the labels
y_true = mlb.fit_transform(df['True Label'])

#refine prediction word list to contain only allowed labels defined by all true labels
df['Predicted Label'] = df['Predicted Label'].apply(lambda x: [label for label in x if label in mlb.classes_])
y_pred = mlb.transform(df['Predicted Label'])

In [76]:
print(mlb.classes_)
print(y_true[0:10])
print(y_pred[0:10])

['announce' 'discussion' 'event' 'job' 'listening' 'question' 'quote'
 'reading' 'recommendation' 'review' 'watching']
[[1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]]
[[1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]]


In [98]:
#evaluate
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Calculate precision, recall, f1_score, support
precision, recall, f1_score, support = precision_recall_fscore_support(y_true, y_pred, average='samples')

accuracy = accuracy_score(y_pred=y_pred,y_true=y_true)

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 score: ', f1_score)
print('Support: ', support)
print('accuracy: ',accuracy)

Precision:  0.6093576965669989
Recall:  0.6782945736434108
F1 score:  0.61343669250646
Support:  None
accuracy:  0.32558139534883723


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [91]:
#Emulating a full run: get a dataset from an artifact create, an evaluation artifact.
import wandb
import pandas as pd
import sys
from pathlib import Path
import os

# Assuming you have a DataFrame named df with your data
#notebook_path = Path(os.getcwd())
#sys.path.append(str(notebook_path.parents[1]))


run = wandb.init(project="testing",job_type="evaluation")

dataset_artifact_id = "common-sense-makers/testing/labeled_data_v0:latest"

dataset_artifact = run.use_artifact(dataset_artifact_id)

#download path to table
a_path = dataset_artifact.download()
table_path = Path(f"{a_path}/labeled_data_table.table.json")
raw_data = json.load(table_path.open())

#put it in a dataframe
x = raw_data["columns"].index('Text')
y = raw_data["columns"].index('True Label')
z = raw_data["columns"].index('Predicted Label')

try:
  rows = [{'Text': raw_data["data"][i][x], 'True Label': raw_data["data"][i][y], 'Predicted Label': raw_data['data'][i][z]} for i in range(len(raw_data["data"]))]
except Exception as e:
  print(f"Exception occurred: {e}")

df = pd.DataFrame(rows)

#print("Got data frame with first raw: ", df)

#Create the evaluation artifact
artifact = wandb.Artifact("prediction_evaluation", type="evaluation")

# Create a wandb.Table from the Pandas DataFrame
table = wandb.Table(dataframe=df)

# Add the wandb.Table to the artifact
artifact.add(table, "prediction_evaluation")

#add the scores as metadata
artifact.metadata.update({'Precision': precision, 'Recall':recall, 'F1 score': f1_score})

# model_info is your model metadata
run.config.update({"model_info": 'model_info'}) 

#log scores as summary of the run
#note that the scores are actually calculated in the cells above.
run.summary["precision"] = precision
run.summary["recall"] = recall
run.summary["f1_score"] = f1_score


# Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()

wandb:   1 of 1 files downloaded.  


Got data frame with first raw:                                                   Text  \
0   Huge congrats to the team on this release. Hop...   
1   Huge congrats to the team on this release. Hop...   
2   "Building a Data Pipeworks for Democratic AI: ...   
3   I'll endorse this and double down: there's no ...   
4   "Ultimately, privacy law’s primary goal should...   
5   The “singularity” to worry about, say ⁦@glenwe...   
6   Join my lab as a postdoc in the Cognitive Scie...   
7   Since writing this, I've found others studying...   
8   Nice perspective paper on fixing activities be...   
9   Some of my dissertation work was recently publ...   
10  A new study published by PNAS adds more detail...   
11  An explanation for why GPT-4 is degrading:\n\n...   
12  This looks very interesting:\n"Secularism as M...   
13  Test sets have 100s or even 1000s of examples\...   
14  From the abstract I can tell this is going to ...   
15  Man, this paper is good: https://philpapers.or...   

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


f1_score,0.61344
precision,0.60936
recall,0.67829


In [97]:
def func(df):
    df1['1'] = df1['1'].apply(lambda x: ['None'] if pd.isnull(x) or x==[] else x) 


df1 = pd.DataFrame([{"1":["da"]},{"1":[]}])

func(df1)   

print(df1["1"])

0      [da]
1    [None]
Name: 1, dtype: object


/var/folders/j3/1_zy_b4s2517g8j_rjjdhd4h0000gn/T/ipykernel_38222/4224999233.py:2: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  df1['1'] = df1['1'].apply(lambda x: ['None'] if pd.isnull(x) or x==[] else x)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score()

## Get CSV from all prediction Artifacts
Create CSV out of all prediction artifacts. Faster than the old script. only returns latest, not all versions

In [2]:
#Select which project
project = "st-app-multi-v1"

In [35]:
import wandb
import pandas as pd
import os
import json
from pathlib import Path

# Initialize wandb
wandb.init()
api = wandb.Api()
# Specify the project and artifact type
project_name = project
TYPE = 'prediction'

# Retrieve all artifacts of the specified type

artifacts = []

collections = [
      coll for coll in api.artifact_type(type_name=TYPE, project=project_name).collections()
  ]


for coll in collections:
    for artifact in coll.artifacts():
        artifacts += [artifact]


# Process artifacts and create a CSV file
csv_data = []
rows = []

for artifact in artifacts:
    #Limit to m entires
    
    # Access the files in the artifact
    a_path = artifact.download()

    table_path = Path(f"{a_path}/predictions.table.json")
    raw_data = json.load(table_path.open())

    row_d = dict(zip(raw_data['columns'],
                     raw_data["data"][0]))
    #print(row_d)
    rows.append(row_d)

df = pd.DataFrame(rows)

print(df)

csv_data.append(df)

# Concatenate all dataframes into one
final_dataframe = pd.concat(csv_data, ignore_index=True)

# Save the concatenated dataframe to a CSV file
output_csv_path = 'output_data.csv'
final_dataframe.to_csv(output_csv_path, index=False)

print(f"CSV file saved at {output_csv_path}")

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downlo

                                                 User  \
0                                                 rtk   
1                                        Nick Vincent   
2                                        Nick Vincent   
3                    p(Alex Kiefer | everything else)   
4                                       Abeba Birhane   
5                                         Nils Gilman   
6                                      Tania Lombrozo   
7                                  Elizabeth McDaniel   
8                                   Kording —-& Lab 🦖   
9                                    Zr. Nabu Kudurru   
10                                 J. Nathan Matias 🦣   
11  rtk - https://twitter.com/ChombaBupe/status/17...   
12                                         Tim Howles   
13                               ♻️ Leshem Choshen ♻️   
14  rtk - https://mastodon.social/@jonny@neuromatc...   
15                                   Mx. Aria Stewart   
16   Filip Piekniewski🌻 🐘:@fili

In [1]:
#Same as the above, only saving directly to an artifact and not CSV
#set constraint on number of rows
m = 100
#Select which project
project = "st-app-multi-v1"

In [5]:
import wandb
import pandas as pd
import os
import json
from pathlib import Path

# Initialize wandb
run = wandb.init(project="evaluation_benchmark",job_type="evaluation")
api = wandb.Api()
# Specify the project and artifact type
project_name = project
TYPE = 'prediction'

# Retrieve all artifacts of the specified type

artifacts = []

collections = [
      coll for coll in api.artifact_type(type_name=TYPE, project=project_name).collections()
  ]


for coll in collections:
    for artifact in coll.versions():
        if len(artifacts)>m:
            break
        artifacts += [artifact]


# Process artifacts and create a CSV file
csv_data = []
rows = []

for artifact in artifacts:
    #Limit to m entires
    
    # Access the files in the artifact
    a_path = artifact.download()

    table_path = Path(f"{a_path}/predictions.table.json")
    raw_data = json.load(table_path.open())

    row_d = dict(zip(raw_data['columns'],
                     raw_data["data"][0]))
    #print(row_d)
    #Check if predicted or true label are empty lists, if so remove dont add the row 
    ind = bool(row_d['Predicted Label'] and row_d['True Label'])
    if ind:
        rows.append(row_d)

df = pd.DataFrame(rows)

#Create the evaluation artifact
artifact = wandb.Artifact("dataset_for_eval", type="dataset")

# Create a wandb.Table from the Pandas DataFrame
table = wandb.Table(dataframe=df)

# Add the wandb.Table to the artifact
artifact.add(table, "labeled_data")

# Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()


wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downlo

In [6]:
# Import necessary libraries
import wandb
import numpy as np
from sklearn.metrics import confusion_matrix

# Initialize wandb
wandb.init(project="testing")

# Assume you have some predictions and true labels
y_true = np.array([2, 0, 2, 2, 0, 1])
y_pred = np.array([0, 0, 2, 2, 0, 2])

# Calculate the confusion matrix using sklearn
cm = confusion_matrix(y_true, y_pred)

# Log the confusion matrix to wandb
wandb.log({"confusion_matrix": wandb.plots.HeatMap(np.unique(y_true), np.unique(y_pred), cm, show_text=True)})


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Visualizing heatmap.
